In [1]:
import json 
import spotipy
import pickle
from spotipy.oauth2 import SpotifyOAuth

import pandas as pd

In [2]:
# read data
df = pd.read_csv('../data/df_final.csv')
df.head()

,name,artist,cluster_main,sub_cluster,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,Se Eu Quiser Falar Com Deus,Gilberto Gil,1,1,0.659467,0.258985,1.000000,0.683418,0.0,0.053496,0.696787,0.000060,0.987706,0.296665,0.456665,1n7JnwviZ7zf0LR1tcGFq7
1,Mulher Eu Sei,Chico César,1,2,0.711263,0.050181,0.363636,0.574692,1.0,0.027362,0.882530,0.000041,0.384284,0.522478,0.432506,3mXqOdlLE1k67WsAxryPFs
2,Rosa Morena,Kurt Elling,1,3,0.651752,0.118982,0.545455,0.537707,1.0,0.017199,0.919679,0.000348,0.095380,0.396105,0.528079,7bSzjzjTkWT2CkIPPdp0eA
3,Desafinado,Stan Getz,1,2,0.447873,0.145983,0.272727,0.588135,1.0,0.047241,0.934739,0.156345,0.098453,0.417858,0.676409,5pstSsK0G4q86gdyy1Spqh
4,Madalena,Maria Gasolina,1,1,0.678201,0.206984,1.000000,0.668576,0.0,0.035627,0.966867,0.420305,0.103575,0.688212,0.342801,33u3dO0JWjerYpFN61J3ug


In [3]:
# specify path to spotify app login json with all correct info
login_file = '../../api_spotify.json'

scope = 'playlist-modify-public'
redirectUri = 'http://localhost:8080/' # as specified in the app settings

with open(login_file) as f:
    login = json.load(f)
    username = login['username']
    client_id = login['client_id']
    client_secret = login['client_secret']

token = SpotifyOAuth(scope=scope,
                     username=username,
                     client_id=client_id,
                     client_secret=client_secret,
                     redirect_uri=redirectUri,
                     )

spotifyObject = spotipy.Spotify(auth_manager = token)

In [7]:
def create_playlist():
# When this code is run, either a browser window will open, or in Google Colab a URL will appear
# This URL will ask you to sign in to Spotify
# After signing in, copy the URL you are redirected to and paste it in the box that pops up
    
    id_list = list(cl.sample(samplesize).id)
    
    # Spotify needs cetain text wrappers around ID numbers to identify the target as a track, a playlist, etc.
    uris = [f'spotify:track:{str(id).strip()}' for id in id_list]
    playlist_collection[playlist_name] = playlist_id

    try:
        spotifyObject.user_playlist_add_tracks(user=username,
                                           playlist_id=playlist_id,
                                           tracks=uris)
    except:
        pass


def update_playlist():
    with open("./playlists.pkl", "rb") as f:
        playlist_collection = pickle.load(f)
        
    uris = [f'spotify:track:{str(id).strip()}' for id in cl.sample(samplesize).id]
    playlist_id = playlist_collection[playlist_name] # We now replace the 5 songs in playlist 0 with 10 different songs
    
    spotifyObject.user_playlist_replace_tracks(user=username,
                                               playlist_id=playlist_id,
                                               tracks=uris)


def save_playlist_in_file():
    # Playlist IDs can be saved for later
    with open("./playlists.pkl", "wb") as f:
        pickle.dump(playlist_collection, f)


In [8]:
playlist_collection = {}

for main in df.cluster_main.unique():
    for sub in df.sub_cluster.unique():
        print(main, sub)
        cl = df.loc[(df.cluster_main == main) & (df.sub_cluster == sub)]
        playlist_name = f'playlist_{main}_{sub}'
        playlist_description= 'this is a test list'

        if cl.shape[0] > 100:
            samplesize = 100 # Here, we simply take songs randomly from each cluster
        else:
            samplesize = cl.shape[0] # in case the cluster has <100 rows

        playlist_id = spotifyObject.user_playlist_create(user=username,
                                                        name=playlist_name,
                                                        public=True,
                                                        description=playlist_description)['id']
        
        #create_playlist() # use only if playlists are not created yet
        update_playlist() # use if playlists already exist

save_playlist_in_file()

1 1
1 2
1 3
1 0
1 4
0 1
0 2
0 3
0 0
0 4
3 1
3 2
3 3
3 0
3 4
2 1
2 2
2 3
2 0
2 4
4 1
4 2
4 3
4 0
4 4
